In [1]:
from vllm import LLM, SamplingParams
from tqdm import tqdm
import datasets
import json
import argparse 

parser = argparse.ArgumentParser(description='Set model name.')  
# parser.add_argument('--model-name', type=str, required=True, help='Name of the model to use')  
# parser.add_argument('--model-path', type=str, default="/home/lidong1/jianglingjie/LLama-Factory/model_checkpoint/huggingface", help='Dir of the model to use') 
# args = parser.parse_args()  
# path_dir = args.model_path
# model_name = args.model_name 

# template = "{{ '<|begin_of_text|>' }}{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ '<|start_header_id|>system<|end_header_id|>\n\n' + system_message + '<|eot_id|>' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ '<|start_header_id|>user<|end_header_id|>\n\n' + content + '<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n' }}{% elif message['role'] == 'assistant' %}{{ content + '<|eot_id|>' }}{% endif %}{% endfor %}"
# template = "{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ system_message + '\n' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ 'Human: ' + content + '\nAssistant:' }}{% elif message['role'] == 'assistant' %}{{ content + '<|end_of_text|>' + '\n' }}{% endif %}{% endfor %}"
model_name = "magpie_8b_2048_default_template"
path_dir = "/mnt/lingjiejiang/textual_aesthetics/exp/saves/magpie_8b_2048_default_template/fullft_lr2e5_e2/sft/checkpoint-500"
# model_name = "Meta-Llama-3.1-8B-Instruct"
# model_name = "Meta-Llama-3.1-8B"
# Create an LLM.
llm = LLM(model=f"{path_dir}")

print(f"model name: {model_name}")
print(f"model_path: {path_dir}/{model_name}")

gen_kwargs_vllm = {
    "max_tokens": 2048,
    "temperature": 0.0,
}
tokenizer = llm.get_tokenizer()
if tokenizer.chat_template is None:
    tokenizer.chat_template = template
    # tokenizer.chat_template = tokenizer.chat_template.replace("<|eot_id|>", tokenizer.eos_token)
    # tokenizer.chat_template
    gen_kwargs_vllm['stop_token_ids'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]
    print(f"tokenizer.chat_template: {tokenizer.chat_template}")
    print("tokenizer is None, use setted template")
else:
    gen_kwargs_vllm['stop_token_ids'] = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|end_of_text|>")]
    print("use original template")
# messages = tokenizer.apply_chat_template(messages, tokenize=False)


sampling_params = SamplingParams(**gen_kwargs_vllm)

eval_set = datasets.load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")["eval"]

def convert_to_message(example):  
    messages = [{"role": "user", "content": example["instruction"]}]  
    example["messages"] = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  
    return example  
eval_set = eval_set.map(convert_to_message)

encoded_inputs = tokenizer.batch_encode_plus(  
    eval_set['messages'],  
    add_special_tokens=False,
) 
input_ids = encoded_inputs['input_ids']  

outputs = llm.generate(prompt_token_ids=input_ids, sampling_params=sampling_params)
outputs_text = [x.outputs[0].text for x in outputs]
eval_set = eval_set.remove_columns(["output"])  # Remove the existing 'output' column if it exists  
eval_set = eval_set.remove_columns(["messages"])
eval_set = eval_set.add_column("output", outputs_text)  
def rename_generator(sample):
    sample['generator'] = f"{model_name}"
    return sample
eval_set = eval_set.map(rename_generator)
eval_set.to_json(f"{model_name}.jsonl", batch_size=128, num_proc=8)

## save data

export_dataset = eval_set
export_data_list = [dict(row) for row in export_dataset]
# print(f"export data length {len(export_data_list)}")
# with open(f'./data/{model_name}.json', 'w', encoding='utf-8') as f:  
#     json.dump(export_data_list, f, ensure_ascii=False, indent=4)  

/home/v-lingjiang/miniconda3/envs/eval/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-03 02:46:35,457	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


WARNING 10-03 02:46:38 arg_utils.py:766] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 10-03 02:46:38 config.py:820] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 10-03 02:46:38 llm_engine.py:174] Initializing an LLM engine (v0.5.4) with config: model='/mnt/lingjiejiang/textual_aesthetics/exp/saves/magpie_8b_2048_default_template/fullft_lr2e5_e2/sft/checkpoint-500', speculative_config=None, tokenizer='/mnt/lingjiejiang/textual_aesthetics/exp/saves/magpie_8b_2048_default_template/fullft_lr2e5_e2/sft/checkpoint-500', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_paral

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:07<00:23,  7.83s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:22<00:24, 12.13s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:35<00:12, 12.44s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:38<00:00,  8.44s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:38<00:00,  9.52s/it]


INFO 10-03 02:47:17 model_runner.py:732] Loading model weights took 14.9888 GB


INFO 10-03 02:47:21 gpu_executor.py:102] # GPU blocks: 28232, # CPU blocks: 2048
INFO 10-03 02:47:23 model_runner.py:1024] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-03 02:47:23 model_runner.py:1028] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
